In [1]:
!pip install gradio
# !pip install git+https://github.com/openai/CLIP.git
!pip install git+https://github.com/salesforce/LAVIS.git
!pip install ftfy
!pip install regex 
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.6 MB 6.9 MB/s 
     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 56 kB 5.2 MB/s 
     |████████████████████████████████| 272 kB 49.2 MB/s 
     |████████████████████████████████| 2.3 MB 54.5 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
     |████████████████████████████████| 106 kB 72.4 MB/s 
     |████████████████████████████████| 84 kB 4.5 MB/s 
     |████████████████████████████████| 55 kB 4.9 MB/s 
     |████████████████████████████████| 213 kB 59.1 MB/s 
     |████████████████████████████████| 64 kB 3.1 MB/s 
     |████████████████████████████████| 80 kB 10.9 MB/s 
     |████████████████████████████████| 68 kB 8.3 MB/s 
     |████████████████████████████████| 68 kB 7.8 MB/s 
     |████████████████████████████████| 68 kB 8.0 MB/s 
     |████████████████████████████████| 46 kB 5.0 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import os
# os.system("pip freeze")
import cv2
from PIL import Image
# import clip
import torch
import math
import numpy as np
import torch
import datetime
import gradio as gr


# Load the open CLIP model
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model, preprocess = clip.load("ViT-B/32", device=device)  

from lavis.models import load_model_and_preprocess

model_name = "clip_feature_extractor"
model_type = "ViT-B-32"
# model_name = "blip_feature_extractor"
# model_type = "base"
device = "cuda" if torch.cuda.is_available() else "cpu"
model, vis_processors, txt_processors = load_model_and_preprocess(name=model_name, model_type=model_type, is_eval=True, device=device)

In [38]:
def inference(video, query, slider):
  # The frame images will be stored in video_frames
  video_frames = []

  # Open the video file
  capture = cv2.VideoCapture(video)
  fps = capture.get(cv2.CAP_PROP_FPS)
  
  current_frame = 1
  num_skip_frames = slider
  # Read the current frame
  ret, frame = capture.read()
  while capture.isOpened() and ret:
    ret, frame = capture.read()
    if current_frame <= num_skip_frames:
      current_frame += 1
      continue
    current_frame = 1
    # print('Read a new frame: ', ret)
    if ret:
      video_frames.append(Image.fromarray(frame[:, :, ::-1]))

  # Print some statistics
  print(f"Frames extracted: {len(video_frames)}")
  
  # You can try tuning the batch size for very large videos, but it should usually be OK
  batch_size = 256
  batches = math.ceil(len(video_frames) / batch_size)
  
  # The encoded features will bs stored in video_features
  # video_features = torch.empty([0, 512], dtype=torch.float16).to(device)
  video_features = []
  
  # Process each batch
  for i in range(batches):
    print(f"Processing batch {i+1}/{batches}")
  
    # Get the relevant frames
    batch_frames = video_frames[i*batch_size : (i+1)*batch_size]
    
    # Preprocess the images for the batch
    # batch_preprocessed = torch.stack([preprocess(frame) for frame in batch_frames]).to(device)
    batch_preprocessed = torch.stack([vis_processors["eval"](frame) for frame in batch_frames]).to(device)
    # batch_preprocessed = vis_processors["eval"](batch_frames).to(device)
    sample = {"image": batch_preprocessed, "text_input": None}
    
    # Encode with CLIP and normalize
    with torch.no_grad():
      if model_name == "clip_feature_extractor":
        batch_features = torch.nn.functional.normalize(model.extract_features(sample), dim=-1)
      else:
        batch_features = model.extract_features(sample, mode="image").image_embeds_proj[:, 0, :]
      # batch_features = model.encode_image(batch_preprocessed)
      # batch_features /= batch_features.norm(dim=-1, keepdim=True)
  
    # Append the batch to the list containing all features
    # video_features = torch.cat((video_features, batch_features))
    video_features.append(batch_features)

  video_features = torch.cat(video_features, dim=0)
  
  # Print some stats
  print(f"Features: {video_features.shape}")
 
 
  search_query=query
  display_heatmap=False
  display_results_count=1

  # Encode and normalize the search query using CLIP
  if not isinstance(search_query, str):
    query_input = vis_processors["eval"](Image.fromarray(search_query)).unsqueeze(0).to(device)
    sample = {"image": query_input, "text_input": None}
    with torch.no_grad():
      if model_name == "clip_feature_extractor":
        query_features = torch.nn.functional.normalize(model.extract_features(sample), dim=-1)
      else:
        query_features = model.extract_features(sample, mode="image").image_embeds_proj[:, 0, :]

      # query_features = model.encode_text(clip.tokenize(search_query).to(device))
      # query_features /= query_features.norm(dim=-1, keepdim=True)
  else:
    query_input = txt_processors["eval"](search_query)
    sample = {"image": None, "text_input": [query_input]}
    with torch.no_grad():
      if model_name == "clip_feature_extractor":
        query_features = torch.nn.functional.normalize(model.extract_features(sample), dim=-1)
      else:
        query_features = model.extract_features(sample, mode="text").text_embeds_proj[:, 0, :]

      # query_features = model.encode_text(clip.tokenize(search_query).to(device))
      # query_features /= query_features.norm(dim=-1, keepdim=True)

  # Compute the similarity between the search query and each frame using the Cosine similarity
  similarities = (100.0 * video_features @ query_features.t())
  print(similarities.cpu().tolist())
  values, best_photo_idx = similarities.topk(display_results_count, dim=0)


  for frame_id in best_photo_idx:
    frame = video_frames[frame_id]
    # Find the timestamp in the video and display it
    frame_id = (frame_id+1)*num_skip_frames
    seconds = round(frame_id.cpu().numpy()[0]/fps)
  return frame,f"Found at {str(datetime.timedelta(seconds=seconds))} with similarity {values.item()}"

In [ ]:
with gr.Blocks() as demo:
  gr.Markdown("Query based on text or image to search event in a video.")
  with gr.Tab("Text Query"):
    with gr.Row():
      with gr.Column():
        text_input = [gr.Video(label="Input Video"), gr.Textbox(label="Text Query"), gr.Slider(0, 100, value=0, step=1, label="Num Skip Frames")]
        text_button = gr.Button("Search")
      with gr.Column():
        text_output = [gr.outputs.Image(type="pil", label="Output"), gr.Textbox(label="Output")]
  with gr.Tab("Image Query"):
    with gr.Row():
      with gr.Column():
        image_input = [gr.Video(label="Input Video"),gr.Image(label="Image Query"), gr.Slider(0, 100, value=0, step=1, label="Num Skip Frames")]
        image_button = gr.Button("Search")
      with gr.Column():
        image_output = [gr.outputs.Image(type="pil", label="Output"), gr.Textbox(label="Output")]

  text_button.click(inference, inputs=text_input, outputs=text_output)
  image_button.click(inference, inputs=image_input, outputs=image_output)

demo.launch(share=True, debug=True)